In [3]:
import requests
import urllib.request
import re
import argparse
import csv
from bs4 import BeautifulSoup
from itertools import zip_longest
#from selenium import webdriver
from lxml import html, etree

# Establecemos una clase que contenga las funciones del scraper para que pueda ser reutilizado
class MoviesToday ():
    

    # Función que descarga la página en la que vamos a realizar el web scraping    
    def _download_web (url):
        
        reply = urllib.request.Request(url, data = None, headers = {
        'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'
        })
        response = urllib.request.urlopen(url, timeout = 10)
        html = response.read()
        return html 
    
    
    # Función que realiza el scraping y devuelve la información que queremos obtener sobre las películas
    def get_movies_info (url):
        
        # Utilizamos Beautiful Soup para realizar el scraping
      
        response = urllib.request.urlopen(url, timeout = 10)
        html = response.read()
        bs = BeautifulSoup(html, 'html.parser')
        
        # Declaramos una lista para cada elemento de información que deseamos obtener sobre cada película
        names = []
        directors = []
        casting = []
        genres = []
        ratings = []
        links = []
        
        # Realizamos el scraping sobre la tabla que contiene las películas en cartelera y su información
        table = bs.findAll('table', id='main-content-table')
        table = table [0]


        # Obtenemos la información de interés que está en una tabla. 
        # Recorremos la tabla extrayendo la información deseada y almacenándola en las listas creadas anteriormente
        for row in table.find_all ('tr'):
    
            for cell in row.find_all ('td'):
        
                for title in cell.find_all ('div', {"class": "mc-right"}):
                    
                    # Extraemos el título 
                    data =title.h3
                    data = data.get_text()
                    names.append(data)
            
                    # Extraemos el director 
                    director = title.find ('div', {"class": "director"})
                    director = director.get_text()
                    director = director.replace('\n', '')
                    directors.append(director)
            
                    # Extraemos el reparto
                    cast = title.find ('div', {"class" : "cast"})
                    cast = cast.get_text()
                    cast = cast.replace('\n', '')
                    casting.append(cast)
            
                    # Extraemos el género
                    genre = title.find ('a', {"class" : "genre"})
                    genre = genre.get_text()
                    genres.append (genre)
           
                    # Extraemos la valoración
                    rating = title.find ('div', {"class" : "avg-rating"})
                    rating = rating.get_text()
                    ratings.append (rating)
             
            # Recorremos de nuevo la tabla y ahora extraemos el enlace de cada película
            for row in table.find_all ('tr'):
    
                for cell in row.find_all ('td'):
        
                    for title in cell.find_all ('div', {"class": "mc-right"}):
            
                        trailer = title.find ('a')
                        link = trailer.get('href')
                        links.append(link)

        # Declaramos una lista que contiene las listas creadas anteriormente y modificamos su formato para exportarla a csv
        info = [names, directors, casting, genres, ratings, links]
        export_data = zip_longest(*info, fillvalue= '')
        
        print ('Scraping successful')
        return export_data
        
        
    # Función que descarga una imagen del póster de cada película
    def _download_images (html):
        
        page = requests.get(url)
        soup = BeautifulSoup (page.content)

        images = []

        # Creamos una lista que contenga las imágenes que vamos a descargar
        for row in soup.find_all ('tr'):
    
            for cell in row.find_all ('td'):

                for img in cell.find_all('div', {"class": "mc-left"}):
            
                    image = img.find('img')
                    image = image.get('src')
                    images.append(image)        

        # Descargamos las imágenes de la lista     
        for image in images:
    
            filename = image.split("/")[-1]
    
            rawImage = requests.get(image, stream = True)
    
            with open (filename, 'wb') as fd:
        
                for chunk in rawImage.iter_content(chunk_size = 1024):
                    fd.write(chunk)
    
        print ('Images downloaded')

    
    # Función que exporta la información de nuestro scraper a un archivo csv
    def data2csv (export_data):
        
        with open ('cinema.csv', 'w', newline = '') as r:
            wr = csv.writer(r)
            wr.writerow(('Names', 'Directors', 'Cast', 'Genre', 'Rating', 'Link'))
            wr.writerows(export_data)
        r.close()
        
        print ('csv created')
        #return 'csv created'

url = 'https://www.filmaffinity.com/uk/topcat.php?id=new_th_uk'       
    
# Para ejecutar el scraper, ejecutamos las funciones creadas anteriormente   
MoviesToday();
MoviesToday._download_images (html);
MoviesToday._download_web(url);
MoviesToday.data2csv (MoviesToday.get_movies_info (url));

Images downloaded
Scraping successful
csv created
